In [9]:
import io
import os
import json
import tempfile
import boto3
from pathlib import Path
from typing import Optional, Union
from urllib.parse import urlparse
import trafilatura
from bs4 import BeautifulSoup
from lxml import etree
from IPython.display import Markdown, display
from datetime import datetime


def _is_s3_uri(path: str) -> bool:
    """Vérifie si le chemin est une URI S3."""
    return path.startswith('s3://') or path.startswith('S3://')


def _read_s3_bytes(s3_uri: str) -> bytes:
    """Lit un fichier S3 et retourne les bytes."""
    parsed = urlparse(s3_uri)
    bucket = parsed.netloc
    key = parsed.path.lstrip('/')
    
    s3_client = boto3.client('s3')
    response = s3_client.get_object(Bucket=bucket, Key=key)
    return response['Body'].read()


def extract_text_from_any_file(
    file_path: Union[str, Path],
    clean_output: bool = True
) -> Optional[str]:
    """
    Extrait le texte de différents types de fichiers (local OU S3).
    Nettoie automatiquement les éléments inutiles si clean_output=True.
    
    Args:
        file_path: Chemin local ou S3 URI (ex: 's3://bucket/key.html')
        clean_output: Si True, nettoie les éléments inutiles (scripts, nav, etc.)
    
    Formats supportés:
    - Texte: txt, md, log, csv, json, xml, html, py, js, java, cpp
    - PDF: PyPDF2
    - DOCX: python-docx
    - Excel: pandas (xlsx/xls)
    - HTML: BeautifulSoup + Trafilatura (nettoyage avancé)
    - Images: pytesseract (si tesseract installé)
    """
    file_path = str(file_path)
    is_s3 = _is_s3_uri(file_path)
    extension = Path(urlparse(file_path).path if is_s3 else file_path).suffix.lower()

    raw_bytes = None
    raw_text = None

    try:
        # ========== Lecture source (S3 ou local) ==========
        if is_s3:
            print(f"   📥 Lecture S3: {file_path}")
            raw_bytes = _read_s3_bytes(file_path)
            size_mb = len(raw_bytes) / (1024 * 1024)
            print(f"   ✅ {size_mb:.2f} MB téléchargé")
        else:
            p = Path(file_path)
            if not p.exists():
                print(f"❌ Fichier introuvable: {file_path}")
                return None
            
            # Formats texte simples
            if extension in ['.txt', '.md', '.log', '.csv', '.json', '.xml', '.html', '.htm', '.py', '.js', '.java', '.cpp']:
                raw_text = p.read_text(encoding='utf-8', errors='ignore')
            else:
                raw_bytes = p.read_bytes()

        # ========== Dispatch par extension avec nettoyage ==========

        # ===== HTML (nettoyage avancé) =====
        if extension in ['.html', '.htm']:
            if raw_text is None:
                raw_text = raw_bytes.decode('utf-8', errors='ignore')
            
            if clean_output:
                print(f"   🧹 Nettoyage HTML avec Trafilatura...")
                
                # Méthode 1: Trafilatura (extrait uniquement le contenu principal)
                text = trafilatura.extract(
                    raw_text,
                    include_comments=False,
                    include_tables=True,
                    no_fallback=False
                )
                
                # Fallback BeautifulSoup si Trafilatura échoue
                if not text or len(text) < 100:
                    print(f"   ⚠️  Fallback BeautifulSoup...")
                    soup = BeautifulSoup(raw_text, 'lxml')
                    
                    # Supprimer éléments inutiles
                    for tag in soup(['script', 'style', 'nav', 'header', 'footer', 
                                    'aside', 'form', 'button', 'iframe', 'noscript']):
                        tag.decompose()
                    
                    text = soup.get_text(separator='\n', strip=True)
                
                # Nettoyer lignes vides multiples
                lines = [line.strip() for line in text.split('\n') if line.strip()]
                return '\n'.join(lines)
            else:
                # Sans nettoyage
                soup = BeautifulSoup(raw_text, 'html.parser')
                return soup.get_text(separator='\n', strip=True)

        # ===== XML (nettoyage namespaces) =====
        elif extension == '.xml':
            if raw_text is None:
                raw_text = raw_bytes.decode('utf-8', errors='ignore')
            
            if clean_output:
                print(f"   🧹 Nettoyage XML...")
                
                size_mb = len(raw_text) / (1024 * 1024)
                
                if size_mb > 100:  # Streaming pour gros fichiers
                    print(f"   ⚡ Mode streaming activé...")
                    
                    # Sauver temporairement
                    with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix='.xml', encoding='utf-8') as tmp:
                        tmp.write(raw_text)
                        tmp_path = tmp.name
                    
                    result = _stream_parse_xml(tmp_path)
                    os.unlink(tmp_path)
                    return result
                else:
                    try:
                        tree = etree.fromstring(raw_text.encode('utf-8'))
                        
                        # Enlever namespaces pour simplifier
                        for elem in tree.iter():
                            if elem.tag.startswith('{'):
                                elem.tag = elem.tag.split('}', 1)[1]
                        
                        # Extraire texte
                        text = ' '.join(tree.itertext())
                        return ' '.join(text.split())  # Nettoyer espaces
                    except Exception:
                        # Si parsing échoue, retourner brut
                        return raw_text
            else:
                return raw_text

        # ===== PDF =====
        elif extension == '.pdf':
            import PyPDF2
            print(f"   📖 Extraction PDF...")
            
            f = io.BytesIO(raw_bytes)
            reader = PyPDF2.PdfReader(f)
            pages = []
            
            for page in reader.pages:
                text = page.extract_text() or ""
                if clean_output:
                    # Supprimer numéros de page et headers/footers communs
                    lines = [line.strip() for line in text.split('\n') if line.strip()]
                    # Filtrer lignes très courtes (souvent des artefacts)
                    lines = [line for line in lines if len(line) > 3]
                    text = '\n'.join(lines)
                pages.append(text)
            
            return "\n\n".join(pages).strip()

        # ===== DOCX =====
        elif extension == '.docx':
            from docx import Document
            print(f"   📝 Extraction DOCX...")
            
            f = io.BytesIO(raw_bytes)
            doc = Document(f)
            
            paragraphs = [para.text.strip() for para in doc.paragraphs if para.text.strip()]
            
            if clean_output:
                # Filtrer paragraphes très courts (souvent artefacts)
                paragraphs = [p for p in paragraphs if len(p) > 2]
            
            return '\n'.join(paragraphs)

        # ===== Excel =====
        elif extension in ['.xlsx', '.xls']:
            import pandas as pd
            print(f"   📊 Extraction Excel...")
            
            f = io.BytesIO(raw_bytes)
            sheets = pd.read_excel(f, sheet_name=None)
            text = []
            
            for sheet_name, df in sheets.items():
                text.append(f"=== Sheet: {sheet_name} ===")
                
                if clean_output:
                    # Supprimer colonnes/lignes entièrement vides
                    df = df.dropna(how='all', axis=0)
                    df = df.dropna(how='all', axis=1)
                
                text.append(df.to_string(index=False))
            
            return "\n\n".join(text)

        # ===== JSON =====
        elif extension == '.json':
            if raw_text is None:
                raw_text = raw_bytes.decode('utf-8', errors='ignore')
            
            try:
                data = json.loads(raw_text)
                if clean_output:
                    # Formater proprement
                    return json.dumps(data, ensure_ascii=False, indent=2)
                else:
                    return raw_text
            except Exception:
                return raw_text

        # ===== CSV =====
        elif extension == '.csv':
            if raw_text is None:
                raw_text = raw_bytes.decode('utf-8', errors='ignore')
            
            if clean_output:
                # Supprimer lignes vides
                lines = [line for line in raw_text.split('\n') if line.strip()]
                return '\n'.join(lines)
            else:
                return raw_text

        # ===== Fichiers code (txt, md, py, etc.) =====
        elif extension in ['.txt', '.md', '.log', '.py', '.js', '.java', '.cpp']:
            if raw_text is None:
                raw_text = raw_bytes.decode('utf-8', errors='ignore')
            
            if clean_output:
                # Supprimer lignes vides excessives
                lines = raw_text.split('\n')
                cleaned = []
                prev_empty = False
                
                for line in lines:
                    is_empty = not line.strip()
                    if is_empty:
                        if not prev_empty:
                            cleaned.append('')
                        prev_empty = True
                    else:
                        cleaned.append(line)
                        prev_empty = False
                
                return '\n'.join(cleaned)
            else:
                return raw_text

        # ===== Images (OCR) =====
        elif extension in ['.png', '.jpg', '.jpeg', '.tiff', '.bmp']:
            from PIL import Image
            import pytesseract
            print(f"   🖼️  OCR avec Tesseract...")
            
            img = Image.open(io.BytesIO(raw_bytes))
            text = pytesseract.image_to_string(img)
            
            if clean_output:
                lines = [line.strip() for line in text.split('\n') if line.strip()]
                return '\n'.join(lines)
            else:
                return text

        # ===== Autres formats =====
        else:
            if raw_text is None and raw_bytes is not None:
                try:
                    raw_text = raw_bytes.decode('utf-8', errors='ignore')
                except Exception:
                    raw_text = None
            
            if raw_text:
                print(f"⚠️  Format {extension} non spécifiquement supporté, retour texte brut")
                return raw_text
            
            print(f"❌ Type de fichier {extension} non supporté")
            return None

    except Exception as e:
        print(f"❌ Erreur lors de l'extraction: {e}")
        return None


def _stream_parse_xml(file_path: str) -> str:
    """Parse XML en streaming pour économiser la mémoire."""
    texts = []
    
    context = etree.iterparse(
        file_path,
        events=('end',),
        tag=etree.Element
    )
    
    for event, elem in context:
        text = (elem.text or '').strip()
        if text and len(text) > 10:
            texts.append(text)
        
        elem.clear()
        while elem.getprevious() is not None:
            del elem.getparent()[0]
    
    return ' '.join(texts)



  

In [10]:
# 3 - fomction permettant de detecter la langue : 
def detect_language(text: str) -> str:
    """
    Détecte la langue d'un texte.
    
    Args:
        text: Texte à analyser
        
    Returns:
        Code de langue (ex: 'en', 'fr', 'es') ou message d'erreur
    """
    if not text or len(text.strip()) < 10:
        return "Erreur: Texte trop court ou vide"
    
    try:
        from langdetect import detect
        return detect(text)
    except ImportError:
        return "Erreur: La bibliothèque langdetect n'est pas installée"
    except Exception as e:
        return f"Erreur detection de langue : {str(e)}"

In [12]:
def translate_and_structure_bedrock_aws(text: str, source_lang: str, chunk_size: int = 15000, overlap: int = 1000) -> str:
    """
    Traduit + structure en Markdown avec Bedrock en découpant en chunks avec overlay.
    
    Args:
        text: Texte à traiter
        source_lang: Code langue source ('en', 'fr', etc.)
        chunk_size: Taille maximale d'un chunk en caractères (réduit à 15k pour respecter limite tokens)
        overlap: Nombre de caractères de chevauchement entre chunks
    
    Returns:
        Texte traduit et structuré en Markdown
    """
    
    client = boto3.client('bedrock-runtime')
    
    # Définir le prompt selon la langue
    if source_lang == 'en':
        task = "Structure this English regulatory document into clean Markdown."
    else:
        task = "Translate this document to English AND structure it as Markdown."
    
    prompt_template = f"""{task}

Rules:
- Accurate translation to English (preserve dates, numbers, names)
- Clean Markdown structure (# ## ###, lists)
- Full content, no summary
- Remove formatting artifacts

Document:
{{chunk}}

Return ONLY Markdown:"""
    
    # Si le texte est court, traiter en un seul appel
    if len(text) <= chunk_size:
        print(f"   📄 Texte court ({len(text):,} chars), traitement en 1 chunk\n")
        
        final_prompt = prompt_template.replace("{chunk}", text)
        
        response = client.converse(
            modelId='anthropic.claude-3-haiku-20240307-v1:0',
            messages=[{"role": "user", "content": [{"text": final_prompt}]}],
            inferenceConfig={
                "temperature": 0.1,
                "maxTokens": 4000  # ✅ MODIFIÉ : limite à 4000 au lieu de 100000
            }
        )
        
        return response["output"]["message"]["content"][0]["text"]
    
    # Sinon, découper en chunks avec overlay
    print(f"   📊 Texte long ({len(text):,} chars)")
    print(f"   ✂️  Découpage en chunks de {chunk_size:,} chars avec overlap de {overlap:,} chars\n")
    
    chunks = []
    start = 0
    chunk_num = 0
    
    while start < len(text):
        end = start + chunk_size
        
        # Prendre le chunk
        chunk = text[start:end]
        
        # Essayer de couper à un paragraphe pour éviter de couper au milieu d'une phrase
        if end < len(text):
            # Chercher le dernier \n\n dans les 500 derniers caractères
            last_para = chunk.rfind('\n\n', -500)
            if last_para > chunk_size - 1000:  # Si trouvé pas trop loin de la fin
                chunk = chunk[:last_para]
                end = start + last_para
        
        chunk_num += 1
        chunks.append((chunk_num, chunk, start, end))
        
        print(f"   📦 Chunk {chunk_num}: {start:,} → {end:,} ({len(chunk):,} chars)")
        
        # Avancer avec overlap
        start = end - overlap
    
    print(f"\n   ✅ {len(chunks)} chunks créés\n")
    
    # Traiter chaque chunk
    translated_chunks = []
    
    for chunk_num, chunk, start_pos, end_pos in chunks:
        print(f"   🔄 Traitement chunk {chunk_num}/{len(chunks)}...", end=" ", flush=True)
        
        final_prompt = prompt_template.replace("{chunk}", chunk)
        
        try:
            response = client.converse(
                modelId='anthropic.claude-3-haiku-20240307-v1:0',
                messages=[{"role": "user", "content": [{"text": final_prompt}]}],
                inferenceConfig={
                    "temperature": 0.1,
                    "maxTokens": 4000  # ✅ MODIFIÉ : limite à 4000
                }
            )
            
            result = response["output"]["message"]["content"][0]["text"]
            translated_chunks.append(result)
            print("✅")
            
        except Exception as e:
            print(f"❌ Erreur: {str(e)}")
            # En cas d'erreur, garder le texte original
            translated_chunks.append(f"\n\n[CHUNK {chunk_num} - Original text kept due to error]\n{chunk}\n\n")
    
    print(f"\n   🔗 Fusion des {len(translated_chunks)} chunks...\n")
    
    # Fusionner les chunks traduits
    final_text = translated_chunks[0]
    
    for i in range(1, len(translated_chunks)):
        chunk_text = translated_chunks[i]
        
        # Enlever les premières lignes qui pourraient être des répétitions
        lines = chunk_text.split('\n')
        # Garder à partir de la 3ème ligne pour éviter les répétitions de l'overlap
        chunk_text = '\n'.join(lines[3:]) if len(lines) > 3 else chunk_text
        
        final_text += "\n\n" + chunk_text
    
    return final_text

In [13]:
# ============================================
# EXEMPLES D'UTILISATION
# ============================================

if __name__ == "__main__":
    
    # ===== Exemple 1: Fichier S3 =====
    print("\n" + "="*60)
    print("📄 Fichier S3 avec nettoyage")
    print("="*60 + "\n")
    
    text = extract_text_from_any_file(
        "s3://csv-file-store-6d12f630/dzd-4giwkqu9tkh07k/alsdvbccweepsg/shared/directives/1.DIRECTIVE (UE) 20192161 DU PARLEMENT EUROPÉEN ET DU CONSEIL.html",
        clean_output=True
    )
    
    if text:
        print(f"✅ {len(text):,} caractères extraits")
        # display(Markdown(text))
        text_in_english_directives = translate_and_structure_bedrock_aws(text, detect_language(text))
        display(Markdown(text_in_english_directives))
        # print(f"📄 Aperçu:\n{text[:300]}...\n")
    
    


📄 Fichier S3 avec nettoyage

   📥 Lecture S3: s3://csv-file-store-6d12f630/dzd-4giwkqu9tkh07k/alsdvbccweepsg/shared/directives/1.DIRECTIVE (UE) 20192161 DU PARLEMENT EUROPÉEN ET DU CONSEIL.html
   ✅ 0.28 MB téléchargé
   🧹 Nettoyage HTML avec Trafilatura...
✅ 80,880 caractères extraits
   📊 Texte long (80,880 chars)
   ✂️  Découpage en chunks de 15,000 chars avec overlap de 1,000 chars

   📦 Chunk 1: 0 → 15,000 (15,000 chars)
   📦 Chunk 2: 14,000 → 29,000 (15,000 chars)
   📦 Chunk 3: 28,000 → 43,000 (15,000 chars)
   📦 Chunk 4: 42,000 → 57,000 (15,000 chars)
   📦 Chunk 5: 56,000 → 71,000 (15,000 chars)
   📦 Chunk 6: 70,000 → 85,000 (10,880 chars)

   ✅ 6 chunks créés

   🔄 Traitement chunk 1/6... ✅
   🔄 Traitement chunk 2/6... ✅
   🔄 Traitement chunk 3/6... ✅
   🔄 Traitement chunk 4/6... ✅
   🔄 Traitement chunk 5/6... ✅
   🔄 Traitement chunk 6/6... ✅

   🔗 Fusion des 6 chunks...



# DIRECTIVE (EU) 2019/2161 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL
of 27 November 2019
amending Council Directive 93/13/EEC and Directives 98/6/EC, 2005/29/EC and 2011/83/EU of the European Parliament and of the Council as regards the better enforcement and modernisation of Union consumer protection rules
(Text with EEA relevance)

## THE EUROPEAN PARLIAMENT AND THE COUNCIL OF THE EUROPEAN UNION,

Having regard to the Treaty on the Functioning of the European Union, and in particular Article 114 thereof,
Having regard to the proposal from the European Commission,
After transmission of the draft legislative act to the national parliaments,
Having regard to the opinion of the European Economic and Social Committee (1),
Acting in accordance with the ordinary legislative procedure (2),
Whereas:

### (1)
Article 169(1) and Article 169(2)(a) of the Treaty on the Functioning of the European Union provide that the Union is to contribute to the attainment of a high level of consumer protection through the measures it adopts pursuant to Article 114 of the Treaty on the Functioning of the European Union. Article 38 of the Charter of Fundamental Rights of the European Union ('the Charter') provides that Union policies are to ensure a high level of consumer protection.

### (2)
Consumer protection law should be effectively enforced throughout the Union. However, the comprehensive fitness check of consumer and marketing law provisions carried out by the Commission in 2016 and 2017 under the Regulatory Fitness and Performance (REFIT) programme concluded that the effectiveness of Union consumer protection law is hampered by a lack of awareness among both traders and consumers, and that the existing redress mechanisms could be more frequently used.

### (3)
The Union has already taken a number of measures to further raise awareness among consumers, traders and consumer law experts, and to improve the enforcement of consumer rights and the remedies available to consumers. However, there remain gaps in national laws regarding truly effective and proportionate penalties to deter and punish intra-Union infringements, the lack of individual remedies available to consumers harmed by infringements of national law transposing Directive 2005/29/EC of the European Parliament and of the Council (3), and the shortcomings of the injunction procedure under Directive 2009/22/EC of the European Parliament and of the Council (4). The revision of the injunction procedure should be addressed by a separate instrument amending and replacing Directive 2009/22/EC.

### (4)
Directives 98/6/EC (5), 2005/29/EC and 2011/83/EU (6) of the European Parliament and of the Council contain obligations requiring Member States to provide for effective, proportionate and dissuasive penalties to address infringements of the national provisions transposing those Directives. In addition, Article 21 of Regulation (EU) 2017/2394 of the European Parliament and of the Council (7) requires Member States to take enforcement measures, including the imposition of penalties, in an effective, efficient and coordinated manner to bring about the cessation or prohibition of widespread infringements or widespread infringements with a Union dimension.

### (5)
The current national rules on penalties vary considerably across the Union. In particular, not all Member States ensure that effective, proportionate and dissuasive fines are imposed on traders who have committed widespread infringements or widespread infringements with a Union dimension. Therefore, it is appropriate to improve the existing rules on penalties in Directives 98/6/EC, 2005/29/EC and 2011/83/EU and, at the same time, to introduce new rules on penalties in Council Directive 93/13/EEC (8).

### (6)
Member States should be able to choose the type of penalties to be imposed and provide for the relevant procedures to impose penalties in the event of infringements of Directives 93/13/EEC, 98/6/EC, 2005/29/EC and 2011/83/EU as amended by this Directive.

### (7)
To facilitate more consistent application of penalties, in particular in the case of intra-Union infringements, widespread infringements and widespread infringements with a Union dimension within the meaning of Regulation (EU) 2017/2394, non-exhaustive and indicative common criteria for the application of penalties should be introduced in Directives 93/13/EEC, 98/6/EC, 2005/29/EC and 2011/83/EU. These criteria should include, for example, the nature, gravity, scale and duration of the infringement, as well as any redress offered by the trader to consumers for the harm caused. The existence of repeated infringements committed by the same perpetrator shows the perpetrator's propensity to commit such infringements and is therefore a significant indication of the gravity of the conduct in question and, consequently, of the need to increase the level of the penalty for effective deterrence. Any financial benefits gained or losses avoided due to the infringement should be taken into account where relevant data is available. Other applicable aggravating or mitigating circumstances in the individual case may also be taken into account.

### (8)
These non-exhaustive and indicative common criteria for the application of penalties may not be relevant for deciding on the penalties applicable to each infringement, in particular as regards minor infringements. Member States should also take into account other generally applicable legal principles relating to the imposition of penalties, such as the principle of ne bis in idem.

### (9)
In accordance with Article 21 of Regulation (EU) 2017/2394, the competent authorities of the Member States concerned by the coordinated action shall, within their competence, take all necessary enforcement measures against the trader responsible for the widespread infringement or the widespread infringement with a Union dimension to bring about the cessation or prohibition of that infringement. Where appropriate, they shall impose penalties, such as fines or periodic penalty payments, on the trader responsible for the widespread infringement or the widespread infringement with a Union dimension. The enforcement measures shall be taken in an effective, efficient and coordinated manner to bring about the cessation or prohibition of the widespread infringement or the widespread infringement with a Union dimension. The competent authorities concerned by the coordinated action shall endeavour to take enforcement measures simultaneously in the Member States concerned by that infringement.

### (10)
In order to ensure that the authorities of the Member States can impose effective, proportionate and dissuasive penalties in the case of a widespread infringement or a widespread infringement with a Union dimension that is subject to coordinated investigation and enforcement measures pursuant to Regulation (EU) 2017/2394, it is appropriate to introduce fines as an element of the penalties for such infringements. To ensure the deterrent effect of fines, Member States should set in their national law the maximum fine for such infringements at a level of at least 4 % of the trader's annual turnover in the Member State or Member States concerned. In some cases, the trader may also be a group of undertakings.

### (11)
In accordance with Articles 9 and 10 of Regulation (EU) 2017/2394, when imposing penalties, due account should be taken, where appropriate, of the nature, gravity and duration of the infringement concerned. The imposition of penalties should be proportionate and in accordance with Union law and national law, including applicable procedural safeguards, as well as with the principles of the Charter. Finally, the penalties imposed should be tailored to the nature of the infringement of Union consumer protection law provisions and the overall actual or potential harm resulting therefrom. The power to impose penalties should be exercised either directly by the competent authorities under their own authority or, where appropriate, by recourse to other competent authorities or to other public authorities, by designating bodies or by requesting the competent courts to take the necessary decision, including, where appropriate, by way of appeal if that request is not successful.

### (12)
Where, as a result of coordinated action under Regulation (EU) 2017/2394, a single competent authority within the meaning of that Regulation imposes a fine on the trader responsible for the widespread infringement or the widespread infringement with a Union dimension, that authority should be able to impose a fine of at least 4 % of the trader's annual turnover in all the Member States concerned by the coordinated enforcement action.

### (13)
Member States should be able to maintain or introduce in their national law higher maximum fines based on the trader's turnover for widespread infringements and widespread infringements with a Union dimension. Member States should also have the possibility to base those fines on the trader's worldwide turnover or to extend the rules on fines to other infringements not covered by the provisions of this Directive related to Article 21 of Regulation (EU) 2017/2394. The obligation to set the fine at a level of at least 4 % of the trader's annual turnover should not apply to any additional rules of the Member States on periodic penalty payments, such as daily fines, in case of non-compliance with a decision, order, interim measure, commitment by the trader or any other measure aimed at bringing the infringement to an end.

### (14)
Rules on penalties should be included in Directive 93/13/EEC in order to strengthen its deterrent effect. Member States are free to decide on the administrative or judicial procedure for the application of penalties for infringements of that Directive. In particular, administrative authorities or national courts could impose penalties when they establish the unfairness of contractual terms, including through court proceedings initiated by an administrative authority. Penalties could also be imposed by administrative authorities or national courts when the trader uses contractual terms that are explicitly defined as unfair in all circumstances under national law, as well as when the trader uses contractual terms that have been found to be unfair in a final binding decision. Member States could provide that administrative authorities also have the power to establish the unfairness of contractual terms. Administrative authorities or national courts could also impose a penalty by the same decision that establishes the unfairness of those terms. Member States could establish appropriate mechanisms to coordinate any action at national level regarding individual redress and penalties.

### (15)
When allocating the revenues from fines, Member States should take into account the strengthening of the general consumer interest as well as other protected public interests.

### (16)
Member States should ensure that consumers harmed by unfair commercial practices have access to remedies in order to eliminate all the effects of those unfair practices. Establishing a clear framework for individual remedies would facilitate the private enforcement of the rules. Consumers should be able to claim compensation for the harm suffered and, where appropriate, a price reduction or the termination of the contract, in a proportionate and effective manner. Member States should be able to maintain or introduce other remedies, such as repair or replacement, for consumers harmed by unfair commercial practices in order to ensure the complete elimination of the effects of such practices. Member States should not be prevented from determining the conditions of application and the effects of remedies for consumers. When applying remedies, the gravity and nature of the unfair commercial practice, the harm suffered by the consumer and other relevant circumstances, such as the trader's fault or the breach of contract, could be taken into account, where appropriate.

### (17)
The fitness check of consumer and marketing law provisions and the parallel evaluation of Directive 2011/83/EU have also identified a number of areas where it is necessary to modernise the existing Union consumer protection rules. In view of the continuous development of digital tools, it is necessary to adapt Union consumer protection law.

### (18)
The ranking of commercial offers or their prominent placement in the context of online search queries conducted by providers of online search functions has a significant impact on consumers.

### (19)
Ranking reflects the relative prominence given to the offers of traders or the relevance attributed to the search results, as presented, organised or communicated by the providers of online search functions, including as a result of the use of algorithmic sequencing, rating or review mechanisms, highlighting, other weighting or a combination thereof.

### (20)
In this regard, Annex I to Directive 2005/29/EC should be amended in order to specify that the practice of providing search results in response to a consumer's online search query without clearly disclosing any paid placement or ranking for which the trader has paid constitutes a misleading commercial practice.


(18) The prominent placement or ranking of commercial offers by providers of online search functionality has a significant impact on consumers.

(19) Ranking reflects the relative priority given to the offers of traders or the relevance attributed to search results, as presented, organised or communicated by providers of online search functionality, including through the use of algorithmic sequencing, rating or scoring mechanisms, highlighting, other tools of emphasis or a combination thereof.

(20) In this regard, Annex I to Directive 2005/29/EC should be amended to specify that it is prohibited for a trader to provide information to a consumer in the form of search results in response to the consumer's online search without clearly indicating the existence of paid advertising or a payment specifically aimed at achieving a higher ranking of products in the search results. Where a trader directly or indirectly pays the provider of online search functionality to benefit from a higher ranking of a product in the search results, that provider should inform consumers in a concise, easily accessible and intelligible manner.

(21) Transparency requirements regarding the main parameters determining ranking are also governed by Regulation (EU) 2019/1150. Similar transparency requirements should therefore be introduced in Directive 2005/29/EC to ensure adequate transparency towards consumers, except in the case of online search engine providers.

(22) Traders that enable consumers to search for goods and services, such as travel, accommodation and leisure activities, offered by different traders or by consumers, should inform consumers of the main default parameters determining the ranking of offers presented to the consumer in the search result and their relative importance compared to other parameters.

(23) The requirement to provide information on the main parameters determining ranking is without prejudice to Directive (EU) 2016/943. Traders should not be required to disclose the detailed functioning of their ranking mechanisms, including algorithms.

(24) When products are offered to consumers on online marketplaces, both the online marketplace provider and the third-party supplier are involved in providing the pre-contractual information required by Directive 2011/83/EU. As a result, consumers using the online marketplace may not clearly know who their contractual partners are and what the implications are for their rights and obligations.

(25) Online marketplaces should be defined for the purposes of Directives 2005/29/EC and 2011/83/EU in the same way as in Regulation (EU) No 524/2013 and Directive (EU) 2016/1148. However, the definition of 'online marketplace' should be updated and made more technologically neutral.

(26) Directives 2005/29/EC and 2011/83/EU should therefore provide for specific information obligations for online marketplaces to inform consumers using such marketplaces of the main parameters determining the ranking of offers and whether they are concluding a contract with a trader or a non-trader, such as another consumer.

(27) Online marketplace providers should inform consumers whether the third party offering the goods, services or digital content is a trader or a non-trader, based on the declaration made to them by the third party. Where the third party offering the goods, services or digital content declares its status as non-trader, online marketplace providers should provide a brief information indicating that the consumer rights under Union consumer protection law do not apply to the concluded contract.

(28) In accordance with Article 15(1) of Directive 2000/31/EC, online marketplace providers should not be required to verify the legal status of third-party suppliers. Online marketplace providers should instead require third-party suppliers on online marketplaces to indicate their status as trader or non-trader for the purposes of consumer protection law and provide this information to online marketplace providers.

(29) Given the rapid evolution of technologies concerning online marketplaces and the need to ensure a high level of consumer protection, Member States should be able to adopt or maintain additional specific measures to that effect. Such provisions should be proportionate, non-discriminatory and without prejudice to Directive 2000/31/EC.

(30) The definitions of the notions of digital content and digital services in Directive 2011/83/EU should be aligned with those in Directive (EU) 2019/770. In case of doubt as to whether a contract is a service contract or a contract for the supply of digital content not supplied on a tangible medium, the rules on the right of withdrawal for services should apply.

(31) Digital content and digital services are often provided online under contracts where the consumer does not pay a price but provides personal data to the trader. Directive 2011/83/EU should apply to contracts for the supply of digital content not supplied on a tangible medium (i.e. the online supply of digital content), whether the consumer pays a price or provides personal data. However, that Directive only applies to service contracts, including digital service contracts, where the consumer pays or undertakes to pay a price. Therefore, that Directive does not apply to digital service contracts where the consumer provides personal data to the trader without paying a price. Given their similarities and the interchangeability of paid digital services and personal data-based digital services, they should be subject to the same rules under that Directive.

(32) It is necessary to ensure consistency between the scope of Directive 2011/83/EU and that of Directive (EU) 2019/770, which applies to contracts for the supply of digital content or digital services for which the consumer provides or undertakes to provide personal data to the trader.

(33) Therefore, the scope of Directive 2011/83/EU should be extended to contracts where the trader supplies or undertakes to supply digital content or a digital service to the consumer, and the consumer provides or undertakes to provide personal data to the trader.


It is appropriate to ensure consistency between the scope of Directive 2011/83/EU and that of Directive (EU) 2019/770, which applies to contracts for the supply of digital content or digital services where the consumer provides or undertakes to provide personal data to the trader.

## Recital (33)

Therefore, the scope of Directive 2011/83/EU should be extended to contracts where the trader supplies or undertakes to supply a digital service to the consumer and the consumer provides or undertakes to provide personal data. Similarly to contracts for the supply of digital content not supplied on a tangible medium, that Directive should apply whenever the consumer provides or undertakes to provide personal data to the trader, unless the personal data provided by the consumer is exclusively processed by the trader to supply the digital content or digital service, and as long as the trader does not process that data for other purposes. Any processing of personal data should be in compliance with Regulation (EU) 2016/679 of the European Parliament and of the Council.

## Recital (34)

In full consistency with Directive (EU) 2019/770, where digital content and digital services are not supplied against the payment of a price, Directive 2011/83/EU should not apply to situations where the trader collects personal data solely for the purpose of complying with legal requirements applicable to the trader. Such situations may include, for example, cases where the consumer's registration is required by applicable laws for security and identification purposes.

## Recital (35)

Directive 2011/83/EU should also not apply to situations where the trader merely collects metadata such as information about the consumer's device or browsing history, unless such a situation is considered a contract under national law. It should also not apply to situations where the consumer, without having concluded a contract with the trader, is exposed to advertisements solely in order to access digital content or a digital service. However, Member States should remain free to extend the application of the rules of Directive 2011/83/EU to such situations or to regulate those situations in another way, which are excluded from the scope of that Directive.

## Recital (36)

The notion of functionality should be understood as referring to the ways in which digital content or a digital service can be used. For example, the absence or existence of technical restrictions, such as protection by digital rights management or regional coding, could affect the ability of the digital content or digital service to perform all the functions intended for it. The notion of interoperability refers to the question of whether the digital content or digital service can function with hardware or software that is different from that with which digital content or digital services of the same type are normally used, and to what extent it can do so. Proper functioning could include, for example, the ability of the digital content or digital service to exchange information with other hardware or software and to use the exchanged information.

## Recital (37)

Article 7(3) and Article 8(8) of Directive 2011/83/EU require traders, respectively for off-premises and distance contracts, to obtain the consumer's prior express consent before the expiry of the withdrawal period. Article 14(4)(a) of that Directive provides for a contractual sanction where this requirement is not met by the trader, namely that the consumer is not liable to pay for the services supplied. The obligation to obtain the consumer's prior express consent is therefore relevant only for services, including digital services, that are supplied against payment of a price. It is therefore necessary to amend Article 7(3) and Article 8(8) so that the obligation for traders to obtain the consumer's prior express consent applies only to contracts for services that subject the consumer to an obligation to pay.

## Recital (38)

Article 16(1)(m) of Directive 2011/83/EU provides for an exception to the right of withdrawal in respect of digital content not supplied on a tangible medium if the consumer has given prior express consent for the performance to begin before the end of the withdrawal period and acknowledges losing the right of withdrawal. Article 14(4)(b) of that Directive provides for a contractual sanction where this requirement is not met by the trader, namely that the consumer is not liable to pay for the digital content consumed. The obligation to obtain the prior express consent and the consumer's acknowledgement is therefore relevant only for digital content supplied against payment of a price. It is therefore necessary to amend Article 16(1)(m) so that the obligation for traders to obtain the prior express consent and the consumer's acknowledgement applies only to contracts that subject the consumer to an obligation to pay.

## Recital (39)

Article 7(4) of Directive 2005/29/EC lays down the information requirements for the invitation to purchase a product at a specified price. Those information requirements already apply at the advertising stage, while Directive 2011/83/EU imposes identical and more detailed information requirements at the subsequent pre-contractual stage (i.e. just before the conclusion of the contract). As a result, traders may be required to provide the same information at the advertising stage (e.g. an online advertisement on a media website) and at the pre-contractual stage (e.g. on the pages of their online shops).

## Recital (40)

The information requirements laid down in Article 7(4) of Directive 2005/29/EC provide for informing the consumer about the trader's complaint handling arrangements. The results of the fitness check of the consumer rights and marketing legislation show that this information is most relevant at the pre-contractual stage, which is governed by Directive 2011/83/EU. The obligation to provide this information in invitations to purchase at the advertising stage under Directive 2005/29/EC should therefore be removed.

## Recital (41)

Article 6(1)(h) of Directive 2011/83/EU requires traders to provide consumers with pre-contractual information on the right of withdrawal, including the model withdrawal form set out in Annex I, Part B, of that Directive. Article 8(4) of that Directive provides for simpler pre-contractual information requirements if the contract is concluded by a means of distance communication that imposes space or time constraints on the presentation of the information, such as by telephone, through a voice assistant or by SMS. The mandatory pre-contractual information to be provided on or through that particular means of distance communication includes the information on the right of withdrawal referred to in Article 6(1)(h). Consequently, it also includes making available the model withdrawal form set out in Annex I, Part B. However, making the withdrawal form available is impossible when the contract is concluded by means such as telephone or through a voice assistant, and it is likely not technically feasible to conveniently provide it through other means of communication referred to in Article 8(4). It is therefore appropriate to exclude the model withdrawal form from the information that traders must provide in all cases on or through the particular means of distance communication used to conclude a contract under Article 8(4).

## Recital (42)

Article 16(1)(a) of Directive 2011/83/EU provides for an exception to the right of withdrawal for service contracts that have been fully performed if the performance has begun with the consumer's prior express consent and the consumer has acknowledged losing the right of withdrawal once the contract has been fully performed by the trader. In contrast, Article 7(3) and Article 8(8) of that Directive, which deal with the trader's obligations in situations where performance of the contract has begun before the expiry of the withdrawal period, only require traders to obtain the consumer's prior express consent, without the consumer acknowledging the loss of the right of withdrawal once the contract has been fully performed. To ensure consistency between these provisions, it is necessary to add, in Article 7(3) and Article 8(8), the obligation for the trader to obtain the consumer's acknowledgement of the loss of the right of withdrawal once the contract has been fully performed if the contract subjects the consumer to an obligation to pay. Furthermore, it is necessary to amend the wording of Article 16(1)(a) to take account of the amendments to Article 7(3) and Article 8(8), according to which the obligation for traders to obtain the consumer's prior express consent and acknowledgement applies only to service contracts that subject the consumer to an obligation to pay. However, Member States should be able to choose not to apply the obligation to obtain the consumer's acknowledgement of the loss of the right of withdrawal once the contract has been fully performed for service contracts where the consumer has expressly requested the trader's visit to carry out repair work.

## Recital (43)

The exception to the right of withdrawal provided for in Article 16(1)(b) of Directive 2011/83/EU should also be considered to apply to contracts for the isolated sale of off-grid energy, as the price of that energy depends on fluctuations on raw materials or energy markets that cannot be controlled by the trader and that may occur during the withdrawal period.

## Recital (44)

Article 14(4) of Directive 2011/83/EU lays down the conditions under which, when exercising the right of withdrawal, the consumer does not bear the costs of the provision of services, of the supply of public utilities and of the supply of digital content not supplied on a tangible medium. Where one of those conditions is met, the consumer is not liable to pay for the service, public utilities or digital content received before the exercise of the right of withdrawal. As regards digital content, one of those non-cumulative conditions, laid down in Article 14(4)(b)(iii), is the absence of confirmation of the contract, including confirmation of the consumer's prior express consent for the performance of the contract to begin before the expiry of the withdrawal period and the consumer's acknowledgement that the right of withdrawal is thereby lost. However, that condition is not among the conditions governing the loss of the right of withdrawal laid down in Article 16(1)(m) of that Directive, which creates uncertainty as to whether consumers can invoke Article 14(4)(b)(iii) when the two other conditions set out in Article 14(4)(b) are met and the right of withdrawal is consequently lost in accordance with Article 16(1)(m). The condition laid down in Article 14(4)(b)(iii) should therefore be added to Article 16(1)(m) in order to enable the consumer to exercise the right of withdrawal where that condition is not met and, consequently, to invoke the rights provided for in Article 14(4).

## Recital (45)

Traders may personalise the price of their offers for specific consumers or groups of consumers through automated decision-making that analyses and profiles the consumer's behaviour to assess the consumer's purchasing power. Where the submitted price is personalised on the basis of automated decision-making, consumers should be clearly informed thereof, so that they can take into account the potential risks involved in their purchasing decision. Therefore, a specific information requirement should be added to Directive 2011/83/EU to indicate to the consumer that the price is personalised on the basis of automated decision-making. This information requirement should not apply where the personalisation of the price is based on factors other than the consumer's purchasing power, such as the volume of the purchase or the consumer's membership of a loyalty programme.


Professionals may personalize the price of their offers for specific consumers or categories of consumers through automated decision-making that analyzes and profiles consumer behavior to assess the consumer's purchasing power. Where the submitted price is personalized based on automated decision-making, consumers should be clearly informed, so that they can take into account the potential risks involved in their purchasing decision. Therefore, a specific information obligation should be added to Directive 2011/83/EU to indicate to the consumer that the price is personalized based on automated decision-making. This information requirement should not apply to techniques such as 'dynamic' or 'real-time' pricing, which involves rapidly and flexibly adjusting prices in response to market demands, where those techniques do not involve personalization based on automated decision-making.

## Recital (46)

Given technological developments, it is necessary to remove the reference to the fax number from the list of means of communication in Article 6(1)(c) of Directive 2011/83/EU, as the fax is now rarely used and largely obsolete.

## Recital (47)

Consumers increasingly rely on reviews and recommendations from other consumers when making purchasing decisions. Therefore, where professionals provide access to consumer reviews of products, they should inform consumers whether there are processes or procedures in place to ensure that the published reviews originate from consumers who have actually used or purchased the products. If such processes or procedures are in place, professionals should provide information on how the checks are carried out and provide consumers with clear information on how the reviews are handled, for example by indicating whether all reviews, whether positive or negative, are published, or whether those reviews have been sponsored or influenced by a contractual relationship with a professional. Furthermore, it should be considered an unfair commercial practice to mislead consumers by stating that product reviews have been submitted by consumers who have actually used or purchased the product when no reasonable and proportionate measures have been taken to ensure that this is the case.

## Recital (48)

The provisions of this Directive on consumer reviews and recommendations are without prejudice to the common and legitimate advertising practice of making exaggerated statements or statements not intended to be taken literally.

## Recital (49)

Professionals should also be prohibited from submitting fake consumer reviews or recommendations, such as 'likes' on social media, or from asking others to do so to promote their products, as well as from manipulating consumer reviews or recommendations, for example by only publishing positive reviews and removing negative reviews. Such practices could also involve extrapolating social recommendations when a user's positive interaction with certain online content is linked or transferred to different but associated content, creating the appearance that the user also has a positive opinion of that associated content.

## Recital (50)

Professionals should be prohibited from reselling to consumers tickets for cultural and sporting events that they have acquired using software bots allowing them to purchase a quantity of tickets exceeding the technical limits imposed by the primary vendor or to circumvent any other technical means put in place by the latter to ensure the accessibility of tickets to all. This prohibition is without prejudice to any other national measures that Member States may take to protect the legitimate interests of consumers as well as to ensure the implementation of cultural policy and to guarantee wide access to cultural and sporting events for all, such as measures regulating the resale price of event tickets.

## Recital (51)

Article 16 of the Charter guarantees the freedom to conduct a business in accordance with Union law and national laws and practices. However, marketing of goods as being identical in various Member States, while having significantly different compositions or characteristics, may mislead consumers and cause them to take a transactional decision that they would not have taken otherwise.

## Recital (52)

Such practices may therefore be considered contrary to Directive 2005/29/EC on the basis of a case-by-case assessment of the relevant elements. In order to facilitate the application of existing Union law by the competent authorities of the Member States on food and consumer rights, guidance on the application of current Union rules to situations of dual quality of food products has been provided in the Commission Communication of 29 September 2017 on the application of EU food and consumer protection law to issues of dual quality of products.

## Recital (53)

However, in the absence of an explicit provision, experience in the application of the rules has shown that consumers, professionals and the competent national authorities do not necessarily know which commercial practices could be contrary to Directive 2005/29/EC. Therefore, that Directive should be amended to ensure legal certainty for both professionals and the authorities responsible for enforcement, by explicitly addressing marketing activities that present a good as identical to a good marketed in other Member States, while that good has significantly different compositions or characteristics.

## Recital (54)

While off-premises sales constitute a legitimate and well-established sales channel, like sales on the professional's business premises and distance sales, certain particularly aggressive or misleading commercial or sales practices in the context of visits to a consumer's home or excursions referred to in Article 2(8) of Directive 2011/83/EU may put consumers under pressure to purchase goods or services they would not otherwise purchase or to purchase goods or services at excessive prices, often involving immediate payment. Such practices often target elderly or other vulnerable consumers. Some Member States consider these practices undesirable and consider it necessary to restrict certain forms and aspects of off-premises sales within the meaning of Directive 2011/83/EU, such as aggressive or misleading commercial or sales practices of a product in the context of unsolicited visits by a professional to a consumer's home or excursions.

## Recital (55)

In accordance with the principle of subsidiarity and in order to facilitate the application of the rules, it should be clarified that Directive 2005/29/EC is without prejudice to the freedom of Member States to adopt national provisions in order to provide a higher level of protection of the legitimate interests of consumers against unfair commercial practices in the context of unsolicited visits by a professional to their home for the purpose of offering or selling products or excursions organized by a professional that have the purpose or effect of promoting or selling products to consumers, where such provisions are justified by consumer protection reasons.

## Recital (56)

As regards aggressive and misleading practices in the context of events organized at locations other than the professional's premises, Directive 2005/29/EC applies without prejudice to the establishment conditions or authorization regimes that Member States may impose on professionals. In addition, that Directive applies without prejudice to national contract law, and in particular rules on the validity, formation or effects of contracts.

## Recital (57)

This Directive should not affect aspects of national contract law that it does not regulate. This Directive should therefore be without prejudice to national contract law regulating, for example, the conclusion or validity of a contract, in particular in the event of lack of consent or unauthorized commercial activity.

## Recital (58)

In order to ensure that citizens have access to up-to-date information on their consumer rights and out-of-court dispute resolution, the online entry point to be set up by the Commission should, to the extent possible, be user-friendly, adapted for portable use, easily accessible and usable by all, including persons with disabilities ('design for all').

## Recital (59)

In accordance with the Joint Political Declaration of the Member States and the Commission of 28 September 2011 on explanatory documents, the Member States have undertaken to accompany, in justified cases, the notification of their transposition measures with one or more documents explaining the relationship between the components of a directive and the corresponding parts of national transposition instruments.

In order to ensure that citizens have access to up-to-date information on their consumer rights and out-of-court dispute resolution, the online entry point to be set up by the Commission should, to the extent possible, be user-friendly, adapted for portable use, easily accessible and usable by all, including persons with disabilities ('design for all').

## Recital (59)
In accordance with the Joint Political Declaration of Member States and the Commission of 28 September 2011 on explanatory documents (17), the Member States have undertaken, in cases where this is justified, to accompany the notification of their transposition measures with one or more documents explaining the relationship between the components of a directive and the corresponding parts of national transposition instruments. In the case of this Directive, the legislator considers the transmission of such documents to be justified.

## Recital (60)
Since the objectives of this Directive, namely better enforcement and modernisation of consumer protection law, cannot be sufficiently achieved by the Member States, but can rather, by reason of the European dimension of the problem, be better achieved at Union level, the Union may adopt measures in accordance with the principle of subsidiarity as set out in Article 5 of the Treaty on European Union. In accordance with the principle of proportionality as set out in that Article, this Directive does not go beyond what is necessary to achieve those objectives.

## Article 1 - Amendment of Directive 93/13/EEC
1. The following article is inserted in Directive 93/13/EEC:
   - Article 8a
     1. Member States shall determine the regime of penalties applicable to infringements of the national provisions adopted pursuant to this Directive and shall take all measures necessary to ensure that they are implemented. Those penalties shall be effective, proportionate and dissuasive.
     2. Member States may restrict such penalties to situations where contractual terms are expressly defined as unfair in all circumstances under national law or where the trader continues to use contractual terms that have been found to be unfair by a final decision in accordance with Article 7(2).
     3. Member States shall ensure that the following non-exhaustive and indicative criteria are taken into account, where appropriate, for the imposition of penalties:
        - (a) the nature, gravity, scale and duration of the infringement;
        - (b) any action taken by the trader to mitigate or remedy the harm suffered by consumers;
        - (c) any previous infringements by the trader;
        - (d) the financial benefits gained or losses avoided by the trader due to the infringement, where the relevant data is available;
        - (e) penalties imposed on the trader for the same infringement in other Member States in cross-border cases where information about those penalties is available through the mechanism established by Regulation (EU) 2017/2394;
        - (f) any other aggravating or mitigating factor applicable to the circumstances of the case.
     4. Without prejudice to paragraph 2 of this Article, Member States shall ensure that where penalties are to be imposed pursuant to Article 21 of Regulation (EU) 2017/2394, they include the possibility of either imposing fines through administrative procedures or engaging in judicial proceedings to impose fines, or both, the maximum amount of such fines being at least 4% of the trader's annual turnover in the Member State or Member States concerned.
     5. In cases where a fine is to be imposed pursuant to paragraph 4, but information on the trader's annual turnover is not available, Member States shall provide for the possibility of imposing fines with a maximum amount of at least EUR 2 million.
     6. By 28 November 2021, Member States shall notify the Commission of the regime and measures referred to in paragraph 1 and shall notify it without delay of any subsequent modification thereof.

## Article 2 - Amendments to Directive 98/6/EC
1. The following article is inserted:
   - Article 6a
     1. Any announcement of a price reduction shall indicate the prior price applied by the trader for a determined period of time before the application of the price reduction.
     2. The prior price shall be the lowest price applied by the trader for a period of not less than 30 days prior to the application of the price reduction.
     3. Member States may provide for different rules for goods that are liable to deteriorate or expire rapidly.
     4. Where the product has been on the market for less than 30 days, Member States may also provide for a shorter period than that referred to in paragraph 2.
     5. Member States may provide that, where the price reduction is progressively increased, the prior price shall be the price without reduction before the first application of the price reduction.
2. Article 8 is replaced by the following:
   - Article 8
     1. Member States shall determine the regime of penalties applicable to infringements of the national provisions adopted pursuant to this Directive and shall take all measures necessary to ensure that they are implemented. Those penalties shall be effective, proportionate and dissuasive.
     2. Member States shall ensure that the following non-exhaustive and indicative criteria are taken into account, where appropriate, for the imposition of penalties:
     3. By 28 November 2021, Member States shall notify the Commission of the regime and measures referred to in paragraph 1 and shall notify it without delay of any subsequent modification thereof.

## Article 3 - Amendments to Directive 2005/29/EC
1. In Article 2, the first subparagraph is amended as follows:
2. In Article 3, paragraphs 5 and 6 are replaced by the following:
   - 5. This Directive shall not prevent Member States from adopting provisions to protect the legitimate interests of consumers against aggressive or misleading commercial practices or sales practices in the context of unsolicited visits by a trader to a consumer's home or excursions organized by a trader with the aim or effect of promoting or selling products to consumers. Such provisions must be proportionate, non-discriminatory and justified on grounds relating to the protection of consumers.
   - 6. Member States shall notify the Commission without delay of any national provisions adopted pursuant to paragraph 5, as well as of any subsequent modification thereof. The Commission shall ensure that this information is easily accessible to consumers and traders on a website created for that purpose.
3. In Article 6(2), the following point is added:
4. Article 7 is amended as follows:
5. The following article is inserted:
   - Article 11a Redress
     1. Consumers who have been harmed by unfair commercial practices shall have access to proportionate and effective remedies, which may include compensation for the harm suffered by the consumer and, where appropriate, a price reduction or the termination of the contract. Member States may determine the conditions for the application and effects of such remedies. Member States may take into account, where appropriate, the gravity and nature of the unfair commercial practice, the harm suffered by the consumer and other relevant circumstances.
     2. Those remedies shall be without prejudice to the application of other remedies available to consumers under Union or national law.
6. Article 13 is replaced by the following:
   - Article 13 Penalties
     1. Member States shall determine the regime of penalties applicable to infringements of the national provisions adopted pursuant to this Directive and shall take all measures necessary to ensure that they are implemented. Those penalties shall be effective, proportionate and dissuasive.
     2. Member States shall ensure that the following non-exhaustive and indicative criteria are taken into account, where appropriate, for the imposition of penalties:
     3. Member States shall ensure that where penalties are to be imposed pursuant to Article 21 of Regulation (EU) 2017/2394, they include the possibility of either imposing fines through administrative procedures or engaging in judicial proceedings to impose fines, or both, the maximum amount of such fines being at least 4% of the trader's annual turnover in the Member State or Member States concerned. Without prejudice to that Regulation, Member States may, for reasons of national constitutional law, restrict the imposition of fines to the following cases:
     4. In cases where a fine is to be imposed pursuant to paragraph 3, but information on the trader's annual turnover is not available, Member States shall provide for the possibility of imposing fines with a maximum amount of at least EUR 2 million.
     5. By 28 November 2021, Member States shall notify the Commission of the regime and measures referred to in paragraph 1 and shall notify it without delay of any subsequent modification thereof.
7. Annex I is amended as follows:

## Article 4 - Amendments to Directive 2011/83/EU
1. In Article 2, the first subparagraph is amended as follows:
2. Article 3 is amended as follows:
3. In Article 5(1), the following subparagraph is added:
4. Article 6 is amended as follows:
5. The following article is inserted:
   - Article 6a Additional specific information requirements for contracts concluded on online marketplaces
     1. Before the consumer is bound by a distance contract or any corresponding offer on an online marketplace, the provider of that online marketplace shall provide the consumer, without prejudice to Directive 2005/29/EC, with the following information in a clear and comprehensible manner and in an appropriate format for the means of distance communication used:
     2. Without prejudice to Directive 2000/31/EC, this Article shall not prevent Member States from imposing additional information requirements on online marketplace providers. Such provisions shall be proportionate, non-discriminatory and justified on grounds relating to the protection of consumers.
6. In Article 7(3), the following subparagraph is added:
7. Article 8 is amended as follows:
8. Article 9 is amended as follows:
9. In Article 10(2), the following subparagraph is added:
10. The following paragraphs are added to Article 13:
    - 4. With regard to the consumer's personal data, the trader shall comply with the applicable obligations under Regulation (EU) 2016/679.
    - 5. The trader shall refrain from using any content, other than the consumer's personal data, which the consumer has provided or created by using the digital content or digital service provided by the trader, except where such content:
    - 6. Except in the situations referred to in paragraph 5, points (a), (b) or (c), the trader shall, upon the consumer's request, make available to the consumer any content, other than the consumer's personal data, which the consumer has provided or created by using the digital content or digital service provided by the trader.
    - 7. The consumer shall have the right to retrieve that digital content without hindrance from the trader, in a reasonable time and in a commonly used and machine-readable format.
    - 8. In the event of withdrawal from the contract, the trader may prevent any further use of the digital content or digital service by the consumer, including by making the digital content or digital service inaccessible to the consumer or deactivating the consumer's user account, without prejudice to

### Informations sur les droits des consommateurs

La Commission veille à ce que les citoyens qui souhaitent obtenir des informations sur leurs droits en tant que consommateurs ou sur le règlement extrajudiciaire des litiges bénéficient d'un point d'accès en ligne, par l'intermédiaire du portail numérique unique établi par le règlement (UE) 2018/1724 du Parlement européen et du Conseil (18), qui leur permet:

a) d'accéder à des informations à jour concernant leurs droits en tant que consommateurs de l'Union de manière claire, compréhensible et aisément accessible; et

b) d'introduire une plainte par l'intermédiaire de la plateforme de règlement en ligne des litiges établie en vertu du règlement (UE) no 524/2013 et auprès du centre compétent du réseau des Centres européens des consommateurs, en fonction des parties concernées.

## Article 6
### Rapport de la Commission et réexamen

Au plus tard le 28 mai 2024, la Commission présente au Parlement européen et au Conseil un rapport sur l'application de la présente directive. Ce rapport contient, en particulier, une évaluation des dispositions de la présente directive concernant:

a) les manifestations organisées sur d'autres sites que les locaux du professionnel; et

b) les cas dans lesquels des biens sont présentés sur le marché comme identiques alors qu'ils ont une composition ou des caractéristiques sensiblement différentes, y compris l'opportunité de soumettre ces cas à des exigences plus strictes, y compris une interdiction visée à l'annexe I de la directive 2005/29/CE, et l'éventuelle nécessité de prévoir des dispositions plus détaillées concernant les informations sur la différenciation des biens.

Ce rapport est accompagné, le cas échéant, d'une proposition législative.

## Article 7
### Transposition

1. Les États membres adoptent et publient, au plus tard le 28 novembre 2021, les dispositions nécessaires pour se conformer à la présente directive. Ils en informent immédiatement la Commission.

Ils appliquent ces dispositions à partir du 28 mai 2022.

Lorsque les États membres adoptent ces dispositions, celles-ci contiennent une référence à la présente directive ou sont accompagnées d'une telle référence lors de leur publication officielle. Les modalités de cette référence sont arrêtées par les États membres.

2. Les États membres communiquent à la Commission le texte des dispositions essentielles de droit interne qu'ils adoptent dans le domaine régi par la présente directive.

## Article 8
### Entrée en vigueur

La présente directive entre en vigueur le vingtième jour suivant celui de sa publication au Journal officiel de l'Union européenne.

## Article 9
### Destinataires

Les États membres sont destinataires de la présente directive.

Fait à Strasbourg, le 27 novembre 2019.

Par le Parlement européen
Le président
D. M. SASSOLI

Par le Conseil
Le président
T. TUPPURAINEN

In [ ]:
# Enregistre le text en MD dans S3 Bucket (peux etre utiliser pour faire le RAG)

folder_uri = "s3://csv-file-store-6d12f630/dzd-4giwkqu9tkh07k/alsdvbccweepsg/UserDataUpload/"
filename = f"rapport_directives_{datetime.now():%Y%m%d_%H%M%S}.md"
# --- Extraire bucket et prefix à partir de l’URI ---
parsed = urlparse(folder_uri)
bucket_name = parsed.netloc
prefix = parsed.path.lstrip("/")  # "notes/projets/"

# --- Construire le chemin complet du fichier ---
file_key = f"{prefix.rstrip('/')}/{filename}"

# --- Upload vers S3 ---
s3 = boto3.client("s3")
s3.put_object(
    Bucket=bucket_name,
    Key=file_key,
    Body=text_in_english_directives.encode("utf-8"),
    ContentType="text/markdown"
)

print(f"✅ Fichier ajouté à : s3://{bucket_name}/{file_key}")